# Our First LLMAgent — Hailstone

## Define the Hailstone Tool

In [1]:
from pydantic import BaseModel

from llm_agents_from_scratch.tools import PydanticFunctionTool


class AlgoParams(BaseModel):
    """Params for next_number."""

    x: int


def next_number(params: AlgoParams) -> int:
    """Generate the next number of the sequence."""
    if params.x % 2 == 0:
        return params.x / 2
    return 3 * params.x + 1


# convert our Python function to a BaseTool
tool = PydanticFunctionTool(next_number)

## Define our backbone LLM

In [2]:
from llm_agents_from_scratch.llms import OllamaLLM

In [3]:
llm = OllamaLLM(model="qwen2.5:3b")

## Define the LLMAgent

In [4]:
from llm_agents_from_scratch import LLMAgent

In [5]:
llm_agent = LLMAgent(
    llm=llm,
    tools=[tool],
)

## Define the Hailstone Task

In [6]:
from llm_agents_from_scratch.data_structures import Task

In [7]:
task_instruction = """
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number 5.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result
"""

In [8]:
task = Task(
    instruction=task_instruction,
)

In [9]:
LOGGING_ENABLED = True

In [10]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

In [11]:
handler = llm_agent.run(task)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: 
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number 5.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result

INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: 
You are given a tool, `next_number`, that generates the following number in the sequence given

In [12]:
handler.done()

True

# See the TaskResult

In [13]:
result = handler.result()

In [14]:
result

TaskResult(task_id='189cffb0-7b94-46ab-9b26-188af931db8e', content='1')

### See the Rollout

In [15]:
print(handler.rollout)

assistant: The current instruction is '
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number 5.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result
'
assistant: I need to make a tool call(s) to `next_number`.
tool: {
    "tool_call": {
        "tool_name": "next_number",
        "arguments": {
            "x": 5
     